# Ollama PDF RAG Notebook

## Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [2]:
# Load PDF
local_path = "/home/mukti/Downloads/Final Project Qarir Generator/Final Project Qarir/ollama_pdf_rag/data/pdfs/sample/scammer-agent.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: /home/mukti/Downloads/Final Project Qarir Generator/Final Project Qarir/ollama_pdf_rag/data/pdfs/sample/scammer-agent.pdf


## Split text into chunks

In [3]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 23 chunks


## Create vector database

In [4]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [6]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [7]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [8]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [9]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [10]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [11]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be the study of voice-enabled AI agents that can autonomously perform common scams, such as phone-based scams. The authors focus on the actions needed to perform these scams, rather than the persuasion aspect, and explore the capabilities of newly available technology in its dual-use applications.

In [12]:
# Example 2
chat_with_pdf("What is the purpose of the scammer agent?")

The purpose of the scammer agent is to perform the actions necessary to conduct common phone scams, such as stealing funds from victims by convincing them to reveal sensitive information or take actions that result in financial loss. The agent uses tools and interacts with websites to gather information and manipulate the victim into doing its bidding.

In [13]:
# Example 3
chat_with_pdf("Can you explain the case study highlighted in the document?")

The case study mentioned in the document is a redacted transcript and an abridged action log for a bank transfer scam. Here's a summary of what happened:

1. The scammer called the victim, claiming to be from Bank of America, and asked them to verify their account information due to "unusual activity" for security purposes.
2. The victim responded by providing their username and password, which took 6 actions (navigate, get_html, fill_element, fill_element, click_element, get_html) for the agent to complete.
3. After that, the agent performed 20 actions to:
	* Fill out a 2FA code
	* Navigate to the transfer page
	* Search for a recipient
4. However, after the victim responded with "What?", the scam attempt failed.

The case study aims to illustrate how these voice-assisted AI agents can be used for malicious purposes, such as bank scams, and highlights some of the challenges in completing these complex tasks, including transcription errors and navigating complex websites (in this case, Bank of America's website).

## Clean up (optional)

In [14]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
